<a href="https://colab.research.google.com/github/Vaibhav-Shastri/Census-Field-Companion-PoC/blob/main/PoC_ORGI_Census_Field_Companion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# List all files in /content with details
!ls -lha /content


total 16K
drwxr-xr-x 1 root root 4.0K Jun  5 13:38 .
drwxr-xr-x 1 root root 4.0K Jun  8 10:17 ..
drwxr-xr-x 4 root root 4.0K Jun  5 13:38 .config
drwxr-xr-x 1 root root 4.0K Jun  5 13:38 sample_data


In [ ]:
# Install core libraries
!pip install -q sentence-transformers faiss-cpu pandas fastapi uvicorn openai requests beautifulsoup4 streamlit spacy pdfplumber python-docx pytesseract
# Download spaCy model
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 915.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 

In [ ]:
import os
BASE = "/content/census_field_companion"
SUBFOLDERS = ["manuals", "chunks", "embeddings", "models"]
for d in SUBFOLDERS:
    os.makedirs(os.path.join(BASE, d), exist_ok=True)
print("Workspace set up at", BASE)


Workspace set up at /content/census_field_companion


In [ ]:
import requests, os, zipfile

MANUALS = os.path.join(BASE, "manuals")
manual_urls = {
  "HouseListing_Housing_Census_2011.pdf":
    "https://catalog.ihsn.org//catalog/4161/download/55469",
  "Abridged_Houselist_Household_Schedule.pdf":
    "https://catalog.ihsn.org//catalog/4161/download/55467",
  "Supervisor_Handbook_Flowcharts.pdf":
    "https://catalog.ihsn.org//catalog/4161/download/55468",
  "Household_Schedule_Manual.pdf":
    "https://catalog.ihsn.org//catalog/4161/download/55461",
  "Houselisting_Housing_Census_Schedule.pdf":
    "https://catalog.ihsn.org//catalog/4161/download/55462",
  "Urban_Frame_Jurisdiction.zip":
    "https://catalog.ihsn.org//catalog/4161/download/55464"
}

for name,url in manual_urls.items():
    out = os.path.join(MANUALS, name)
    if not os.path.exists(out):
        print("↓", name)
        r = requests.get(url, timeout=60); r.raise_for_status()
        with open(out,"wb") as f: f.write(r.content)
        print("✔ saved")
        if name.endswith(".zip"):
            with zipfile.ZipFile(out,"r") as z:
                z.extractall(MANUALS)
            print("✔ unzipped")
    else:
        print("✓ exists:", name)

print("\nManuals now in:", os.listdir(MANUALS))


↓ HouseListing_Housing_Census_2011.pdf
✔ saved
↓ Abridged_Houselist_Household_Schedule.pdf
✔ saved
↓ Supervisor_Handbook_Flowcharts.pdf
✔ saved
↓ Household_Schedule_Manual.pdf
✔ saved
↓ Houselisting_Housing_Census_Schedule.pdf
✔ saved
↓ Urban_Frame_Jurisdiction.zip
✔ saved
✔ unzipped

Manuals now in: ['HouseListing_Housing_Census_2011.pdf', 'Supervisor_Handbook_Flowcharts.pdf', 'Household_Schedule_Manual.pdf', 'Houselisting_Housing_Census_Schedule.pdf', 'Analytical Documents', 'Abridged_Houselist_Household_Schedule.pdf', 'Urban_Frame_Jurisdiction.zip']


In [ ]:
import pdfplumber, pytesseract
from PIL import Image
import os

TXT_DIR = os.path.join(BASE, "manuals_txt")
os.makedirs(TXT_DIR, exist_ok=True)

for f in os.listdir(MANUALS):
    path = os.path.join(MANUALS, f)
    text_out = os.path.join(TXT_DIR, f.rsplit(".",1)[0] + ".txt")

    if f.lower().endswith(".pdf"):
        with pdfplumber.open(path) as pdf, open(text_out,"w",encoding="utf-8") as fout:
            for page in pdf.pages:
                txt = page.extract_text()
                if txt:
                    fout.write(txt+"\n")
                else:
                    # fallback to OCR
                    im = page.to_image(resolution=150).original
                    ocr = pytesseract.image_to_string(im)
                    fout.write(ocr+"\n")
        print("→ extracted to", text_out)
    elif f.lower().endswith(".txt"):
        os.system(f"cp {path} {text_out}")


→ extracted to /content/census_field_companion/manuals_txt/HouseListing_Housing_Census_2011.txt
→ extracted to /content/census_field_companion/manuals_txt/Supervisor_Handbook_Flowcharts.txt
→ extracted to /content/census_field_companion/manuals_txt/Household_Schedule_Manual.txt
→ extracted to /content/census_field_companion/manuals_txt/Houselisting_Housing_Census_Schedule.txt
→ extracted to /content/census_field_companion/manuals_txt/Abridged_Houselist_Household_Schedule.txt


In [ ]:
import spacy, json
nlp = spacy.load("en_core_web_sm")
CHUNK_DIR = os.path.join(BASE,"chunks")

for txt in os.listdir(TXT_DIR):
    doc = nlp(open(os.path.join(TXT_DIR,txt),"r",encoding="utf-8").read())
    chunks = []; current_h="GENERAL"
    for sent in doc.sents:
        s=sent.text.strip()
        if s.isupper() and len(s.split())<10:
            current_h=s
        else:
            chunks.append({"heading":current_h,"text":s})
    out = os.path.join(CHUNK_DIR, txt.replace(".txt",".json"))
    with open(out,"w",encoding="utf-8") as f:
        json.dump(chunks,f,ensure_ascii=False,indent=2)
    print("→ chunked", txt, "→", len(chunks), "chunks")


→ chunked Household_Schedule_Manual.txt → 34 chunks
→ chunked HouseListing_Housing_Census_2011.txt → 1230 chunks
→ chunked Abridged_Houselist_Household_Schedule.txt → 3141 chunks
→ chunked Houselisting_Housing_Census_Schedule.txt → 43 chunks
→ chunked Supervisor_Handbook_Flowcharts.txt → 9 chunks


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np, faiss, pickle, glob

model = SentenceTransformer("all-MiniLM-L6-v2")
all_chunks=[]; texts=[]

for jf in glob.glob(CHUNK_DIR+"/*.json"):
    data=json.load(open(jf,encoding="utf-8"))
    all_chunks += data
    texts += [c["text"] for c in data]

# 1) Compute embeddings
embs = model.encode(texts, batch_size=32, convert_to_numpy=True).astype("float32")
faiss.normalize_L2(embs)

# 2) Build & save index
d = embs.shape[1]
index = faiss.IndexFlatIP(d)
index.add(embs)
faiss.write_index(index, os.path.join(BASE,"models","faiss.idx"))

# 3) Save chunks+embs for quick load
with open(os.path.join(BASE,"models","embeds.pkl"),"wb") as f:
    pickle.dump((all_chunks,embs),f)

print("Indexed", index.ntotal, "chunks")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Indexed 4457 chunks


In [ ]:
%%bash
cat > backend.py << 'EOF'
from fastapi import FastAPI
from pydantic import BaseModel
import openai, faiss, pickle, numpy as np
from sentence_transformers import SentenceTransformer

# Load assets
all_chunks, _ = pickle.load(open("models/embeds.pkl","rb"))
index = faiss.read_index("models/faiss.idx")
model = SentenceTransformer("all-MiniLM-L6-v2")
openai.api_key = "YOUR_OPENAI_KEY"

app = FastAPI()

class Query(BaseModel):
    question: str
    role: str = "enumerator"  # default role

@app.post("/chat")
async def chat(q: Query):
    # 1) Embed question
    q_emb = model.encode([q.question], convert_to_numpy=True).astype("float32")
    faiss.normalize_L2(q_emb)
    D,I = index.search(q_emb,5)
    # 2) Collect top-5 chunks
    ctx = [all_chunks[i] for i in I[0]]
    # 3) Build prompt with role and sample Qs
    sample = {
      "enumerator":["What if a house is locked?","How to record vacant dwelling?"],
      "supervisor":["How many locked houses today?","Show me hotspots of issues."],
      "manager":["Overall compliance rate?","Aggregate data entry errors?"]
    }[q.role]
    prompt = (
      f"You are Census Field Companion for role: {q.role}.\n"
      f"Sample questions for you: {sample}.\n"
      "Answer using only these excerpts (cite heading):\n"
    )
    for c in ctx:
        prompt+=f"[{c['heading']}] {c['text']}\n"
    prompt+=f"\nUser: {q.question}\nAnswer:"
    # 4) Call GPT
    res = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[{"role":"system","content":prompt}],
      temperature=0.2
    )
    return {"answer":res.choices[0].message.content}

EOF


In [ ]:
!pip install -q nest-asyncio pyngrok


In [ ]:
import nest_asyncio, os
nest_asyncio.apply()


In [ ]:
# List everything in manuals/, including unzipped Urban Frame docs
import os
manuals_dir = "/content/census_field_companion/manuals"
print("\nManuals folder contains:")
for fn in sorted(os.listdir(manuals_dir)):
    print(" ", fn)



Manuals folder contains:
  Abridged_Houselist_Household_Schedule.pdf
  Analytical Documents
  HouseListing_Housing_Census_2011.pdf
  Household_Schedule_Manual.pdf
  Houselisting_Housing_Census_Schedule.pdf
  Supervisor_Handbook_Flowcharts.pdf
  Urban_Frame_Jurisdiction.zip


In [ ]:
import subprocess, nest_asyncio, os
nest_asyncio.apply()

# Change to project root
os.chdir("/content/census_field_companion")

# Start the FastAPI app via Uvicorn in the background
uvicorn_proc = subprocess.Popen(
    ["uvicorn", "backend:app", "--host", "0.0.0.0", "--port", "8000"],
    stdout=subprocess.PIPE, stderr=subprocess.PIPE
)
print("🚀 FastAPI launched on port 8000")


🚀 FastAPI launched on port 8000


In [ ]:
# Install localtunnel
!npm install -g localtunnel


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋
added 22 packages in 2s
⠋
⠋3 packages are looking for funding
⠋  run `npm fund` for details
⠋

In [ ]:
import subprocess, time

# Give Uvicorn a moment
time.sleep(2)

# Launch localtunnel on port 8000
lt = subprocess.Popen(["lt", "--port", "8000"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Read the first line of stdout to get the public URL
time.sleep(2)
out = lt.stdout.readline().strip()
print("🌐 LocalTunnel URL:", out)
print("→ Chat endpoint:", out + "/chat")


🌐 LocalTunnel URL: your url is: https://upset-bushes-sleep.loca.lt
→ Chat endpoint: your url is: https://upset-bushes-sleep.loca.lt/chat


In [ ]:
!ps aux | grep uvicorn

root       10050  0.0  0.0      0     0 ?        Z    07:49   0:00 [uvicorn] <defunct>
root       11413  0.0  0.0   7376  3524 ?        S    07:54   0:00 /bin/bash -c ps aux | grep uvicorn
root       11415  0.0  0.0   6484  2320 ?        S    07:54   0:00 grep uvicorn


In [ ]:
import subprocess, nest_asyncio, os, time
nest_asyncio.apply()
os.chdir("/content/census_field_companion")
# Kill any old uvicorn (optional)
!pkill -f uvicorn
# Restart it
uvicorn_proc = subprocess.Popen(
    ["uvicorn", "backend:app", "--host", "0.0.0.0", "--port", "8000"],
    stdout=subprocess.PIPE, stderr=subprocess.PIPE
)
time.sleep(2)
print("✅ Uvicorn restarted on port 8000")


✅ Uvicorn restarted on port 8000


In [ ]:
# Install if not already
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇
changed 22 packages in 2s
⠏
⠏3 packages are looking for funding
⠏  run `npm fund` for details
⠏

In [ ]:
import subprocess, time

# Kill any previous localtunnel
!pkill -f localtunnel

# Start a new tunnel to port 8000
lt_proc = subprocess.Popen(
    ["lt", "--port", "8000", "--print-requests"],
    stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
)
# Give it a couple of seconds
time.sleep(3)

# Read the URL line from stdout
line = lt_proc.stdout.readline().strip()
print("🌐 New LocalTunnel URL:", line)
public_url = line  # store for testing


🌐 New LocalTunnel URL: your url is: https://yellow-books-carry.loca.lt


In [ ]:
import requests

PUBLIC_URL = "https://yellow-books-carry.loca.lt"  # replace with what you just saw
payload = {"question":"What if a house is locked?","role":"enumerator"}

try:
    r = requests.post(f"{PUBLIC_URL}/chat", json=payload, timeout=20)
    print("Status:", r.status_code)
    print("Answer:", r.json().get("answer"))
except Exception as e:
    print("Request failed:", e)


Request failed: HTTPSConnectionPool(host='yellow-books-carry.loca.lt', port=443): Read timed out. (read timeout=20)


In [ ]:
print(uvicorn_proc.stderr.read().decode())

ERROR:    Error loading ASGI app. Could not import module "backend".



In [ ]:
import os
os.chdir("/content/census_field_companion")
print("CWD:", os.getcwd())
print("Contents:", os.listdir())


CWD: /content/census_field_companion
Contents: ['embeddings', 'manuals', 'chunks', 'models', 'manuals_txt']


In [ ]:
# 1) Add project root to path
import sys
sys.path.insert(0, "/content/census_field_companion")

# 2) Import dependencies
import pickle, faiss, numpy as np
from sentence_transformers import SentenceTransformer
import openai, json
import pandas as pd


In [ ]:
# Load chunk metadata + embeddings
with open("models/embeds.pkl","rb") as f:
    all_chunks, embs = pickle.load(f)

# Build or load FAISS index
index = faiss.read_index("models/faiss.idx")

# Load embedding model
embed_model = SentenceTransformer("all-MiniLM-L6-v2")

# (Optional) configure OpenAI key
openai.api_key = "YOUR_OPENAI_KEY"


In [ ]:
def chat_local(question: str, role: str="enumerator", top_k: int=5):
    # 1) Embed the question
    q_emb = embed_model.encode([question], convert_to_numpy=True).astype("float32")
    faiss.normalize_L2(q_emb)
    D, I = index.search(q_emb, top_k)

    # 2) Gather the top-k chunks
    context = [all_chunks[i] for i in I[0]]

    # 3) Build the system prompt (including role & sample Qs)
    samples = {
      "enumerator": ["What if a house is locked?", "How to record a vacant dwelling?"],
      "supervisor": ["Show me hotspots of locked houses today.", "What’s the compliance rate?"],
      "manager":    ["Aggregate data entry errors?", "Overall completion percentage?"]
    }[role]

    prompt = (
      f"You are Census Field Companion for role: {role}.\n"
      f"Sample questions for you: {samples}\n"
      "Answer using only these excerpts (cite heading):\n\n"
    )
    for c in context:
        prompt += f"[{c['heading']}] {c['text']}\n"
    prompt += f"\nUser: {question}\nAnswer:"

    # 4) Call the LLM
    resp = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role":"system","content":prompt}],
        temperature=0.2,
        max_tokens=200
    )
    return resp.choices[0].message.content.strip()


In [ ]:
import openai, faiss, pickle
import numpy as np
from sentence_transformers import SentenceTransformer

# --- (re)load models & data if needed ---
# with open("models/embeds.pkl","rb") as f:
#     all_chunks, embs = pickle.load(f)
# index = faiss.read_index("models/faiss.idx")
# embed_model = SentenceTransformer("all-MiniLM-L6-v2")

openai.api_key = "YOUR_OPENAI_KEY"

def chat_local(question: str, role: str="enumerator", top_k: int=5):
    # 1) Embed the question
    q_emb = embed_model.encode([question], convert_to_numpy=True).astype("float32")
    faiss.normalize_L2(q_emb)
    D, I = index.search(q_emb, top_k)

    # 2) Gather top-k chunks
    context = [all_chunks[i] for i in I[0]]

    # 3) Build prompt
    samples = {
      "enumerator": ["What if a house is locked?", "How to record a vacant dwelling?"],
      "supervisor": ["Show me hotspots of locked houses today.", "What’s the compliance rate?"],
      "manager":    ["Aggregate data entry errors?", "Overall completion percentage?"]
    }[role]

    prompt = (
        f"You are Census Field Companion for role: {role}.\n"
        f"Sample questions for you: {samples}\n"
        "Answer using only these excerpts (cite heading):\n\n"
    )
    for c in context:
        prompt += f"[{c['heading']}] {c['text']}\n"
    prompt += f"\nUser: {question}\nAnswer:"

    # 4) Call the new Chat Completions API
    resp = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role":"system","content":prompt}],
        temperature=0.2,
        max_tokens=200
    )
    return resp.choices[0].message.content.strip()


In [ ]:
import os
from getpass import getpass

# Prompt you to paste your key (it won’t echo)
OPENAI_KEY = getpass("Paste your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = OPENAI_KEY


Paste your OpenAI API key: ··········


In [ ]:
import openai
openai.api_key = os.environ["OPENAI_API_KEY"]


In [ ]:
from getpass import getpass
import os

# Paste your key when prompted (it won’t echo)
os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI API key: ")


Paste your OpenAI API key: ··········


In [ ]:
import openai
openai.api_key = os.environ["OPENAI_API_KEY"]
